In [0]:
import delta

In [0]:
df_full = spark.read.format("parquet").load("/Volumes/raw/upcell/full-load/clientes/clientes.parquet")
(df_full.coalesce(1)
        .write
        .format("delta")
        .mode("overwrite")
        .saveAsTable("bronze.upcell.clientes"))

In [0]:
(spark.read
    .format("parquet")
    .load("/Volumes/raw/upcell/cdc/clientes")
    .createOrReplaceTempView("clientes"))

In [0]:
query =  """
    select *  
    from clientes
    qualify row_number() over(partition by idCliente order by DtAtualizacao desc) = 1

"""
df_cdc_unique = spark.sql(query)
df_cdc_unique.display()

In [0]:
bronze = delta.DeltaTable.forName(spark, "bronze.upcell.clientes")
bronze

In [0]:
(bronze.alias("b") 
  .merge(df_cdc_unique.alias("d"),  "b.idCliente = d.idCliente") 
  .whenMatchedDelete(condition = "d.OP = 'D'")   # Delete se for tipo D
  .whenMatchedUpdateAll(condition = "d.OP ='A'")  # Atualiza se for tipo A
  .whenNotMatchedInsertAll(condition = "d.OP = 'I' OR d.OP = 'U'")  # Insere se for tipo I
  .execute()
)

In [0]:
%sql
select*from bronze.upcell.clientes